ForneyLab.jl basics tutorial
============================
This is the first tutorial in a series of practical demos that showcase ForneyLab in action. In this demo we show how to build a basic factor graph and how to calculate a sum-product message. For clarification on the basic types and functionality of the ForneyLab.jl toolbox please consult the [ForneyLab documentation](http://spsbrats.github.io/ForneyLab/documentation). We assume the reader has basic knowledge of message passing algorithms and factor graphs. 

If you are following this tutorial online you do not need to install Julia. However, for working with the ForneyLab.jl toolbox on your own machine and executing the demo code you need to install Julia (v.0.3). Installation instructions can be found on the Julia webpage and the `README.md` file.

Graph contruction example
-------------------------
---
Before starting any project, we need to include the ForneyLab toolbox.

In [1]:
using ForneyLab

We want to create a simple factor graph where an incoming message is multiplied with a constant factor `A`.

```
node1  node2    node3
 [N]--->[A]----->[T]
            out
```

First we initialize the Gaussian source node on the left by constructing a `PriorNode` and setting its value to a Gaussian probability distribution. We set the distribution mean and variance to 1. We also assign an (optional) name. After construction we inspect the node and its value. We see that the node's value is exactly what we set it to be.

In [2]:
PriorNode(GaussianDistribution(m=1.0, V=1.0), id=:prior_node)
show(node(:prior_node))
show(node(:prior_node).value)

TerminalNode with id prior_node


N(m=[1.00], V=[[1.00]])


There is also a more compact way to access nodes.

In [3]:
show(n(:prior_node))

TerminalNode with id prior_node


Next, we initialize the multiplication by `A` by constructing a `FixedGain` node. Here, we set the multiplication factor to 2. We inspect the node and its multiplication factor and see everything is in order.

In [4]:
FixedGainNode(2.0, id=:factor_multiplication_node)
show(n(:factor_multiplication_node))
show(n(:factor_multiplication_node).A)

FixedGainNode with id factor_multiplication_node


[

2.0]

Lastly, we create the `TerminalNode` that completes out model.

In [5]:
TerminalNode(id=:end_node)

TerminalNode with id end_node


The next step is to connect the nodes with edges. Edges can optionally be given an id. When no id is provided, ForneyLab combines the ids of the tail and head (in that order).

In [6]:
Edge(n(:prior_node), n(:factor_multiplication_node).i[:in])
Edge(n(:factor_multiplication_node).i[:out], n(:end_node), id=:y);

Edges can be accessed through their ids, in verbose and compact notation.

In [7]:
show(edge(:prior_node_factor_multiplication_node))
show(e(:y))

Edge with id prior_node_factor_multiplication_node from prior_node.i[:out] to factor_multiplication_node.i[:in].


Edge with id y from factor_multiplication_node.i[:out] to end_node.i[:out].


That's it, the graph is ready. We can inspect our work by invoking `draw()`:

In [8]:
draw() # Visualizes the current graph

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 2119643618463187072 -->
 2119643618463187072 
 
 prior_node 
 
<!-- 1747972883356198987 -->
 1747972883356198987 
 
 FixedGainNode 
 factor_multiplication_node 
 
<!-- 2119643618463187072->1747972883356198987 -->
 2119643618463187072->1747972883356198987 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 1 in 
 1 out 
 
<!-- 318913969658651498 -->
 318913969658651498 
 
 end_node 
 
<!-- 1747972883356198987->318913969658651498 -->
 1747972883356198987->318913969658651498 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 1 out 
 2 out

Message calculation example
---------------------------
---
Now suppose we want to calculate the sum-product message from the multiplication node towards the terminal node. We can do this by  defining a sum-product algorithm that implements a message passing schedule towards the `out` interface of the multiplication node. 

In [9]:
algo = SumProduct.Algorithm(n(:factor_multiplication_node).i[:out])
show(algo.fields[:schedule])

Message passing schedule (entry: node [interface], rule)


------------------------------------------------------
1: TerminalNode prior_node [1:out], sumProduct! 
2: FixedGainNode factor_multiplication_node [2:out], sumProduct! 


Now we can run the algorithm in order to calculate our message of interest.

In [10]:
run(algo)

Message{GaussianDistribution} with payload N(m=[2.00], V=[[4.00]])

